In [ ]:
#COMP4121 MODEL 3 INFERENCE

import os
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import torch
import torch.nn as nn
from torchvision.utils import make_grid
from torchvision.utils import save_image
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import random
import torch.optim as optim
import time
from PIL import Image
import pandas as pd

train_image_path = '../input/resized-plant2021/img_sz_256/' #uses smaller version of dataset for efficiency
test_image_path = '../input/plant-pathology-2021-fgvc8/test_images/'
train_df_path = '../input/plant-pathology-2021-fgvc8/train.csv'
test_df_path = '../input/plant-pathology-2021-fgvc8/sample_submission.csv'

train_df = pd.read_csv(train_df_path)

classes_ = train_df.labels.unique().tolist()

class PlantDataSet(Dataset):
    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.transform = transform
        self.all_imgs = os.listdir(main_dir)

    def __len__(self):
        return len(self.all_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.all_imgs[idx])
        image = Image.open(img_loc).convert("RGB")
        tensor_image = self.transform(image)
        label = classes_.index(train_df.iloc[idx]['labels'])
        return tensor_image, label
    
transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomResizedCrop(256),
     transforms.ToTensor()])

BATCH_SIZE = 256

import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(59536, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 12)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

net = Net()
net.load_state_dict(torch.load("../input/plant-cnn/cifar_net.pth"))

my_dataset = PlantDataSet(test_image_path, transform=transform)
testloader = DataLoader(my_dataset , batch_size=256)

preds = []

with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        for pred in predicted:
            preds.append(classes_[pred])
        
test_df = pd.read_csv(test_df_path)
test_df['labels'] = preds
test_df.to_csv('submission.csv', index=False)